In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
from torch.utils.data import Dataset, DataLoader,random_split
from torchvision import models
import time
import glob
import PIL
import os
from PIL import Image

In [2]:
DATA_PATH='RSI-CB256/data'

# Defining image transformations
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Resize((64,64)),
    # Normalize the image
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])]
    )


In [3]:
# Defining hyperparameters
batch_size = 32
num_epochs = 10
learning_rate = 0.0003

In [4]:
data = datasets.ImageFolder(DATA_PATH, transform = transform) # Fetching dataset from folder

# train-test split
train_size = int(0.80 * len(data)) 
test_size = len(data) - train_size
train_dataset, test_dataset = random_split(data, [train_size, test_size])


print(f"Dataset has {len(data)} data points.")
print(f"Train Dataset has {len(train_dataset)} data points")
print(f"Test Dataset has {len(test_dataset)} data points.") 

Dataset has 5631 data points.
Train Dataset has 4504 data points
Test Dataset has 1127 data points.


In [5]:
train_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)
test_loader = DataLoader(test_dataset, batch_size= batch_size, shuffle= False)

In [6]:
# # Visualizing data distrubitons accordin to classes
# class_names_train = os.listdir(DATA_PATH)

# image_count = {}
# for i in class_names_train:
#     image_count[i] = len(os.listdir(os.path.join(DATA_PATH,i))) - 1

    
# # Plotting Distribution of Each Classes
# fig1, ax1 = plt.subplots(figsize=(10, 10), subplot_kw=dict(aspect="equal"))
# ax1.pie(image_count.values(),
#         labels = image_count.keys(),
#         shadow=True,
#         textprops=dict(color="black"),
#         autopct = '%1.1f%%',
#         startangle=90)
# plt.title("Dataset Classes Distribution Chart", color='black')
# plt.show()
# print(image_count)

In [7]:
# Defining Model
model = models.resnet18()

for param in model.parameters():
    param.require_grad = False
    
fc = nn.Sequential(
    nn.AdaptiveMaxPool2d(1), 
    nn.Flatten(), 
    nn.Dropout(0.2),
    nn.Linear(512, 5)
)

model.classifier = fc
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
model.to(device)

# Define the loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Device: cuda:0


In [8]:
total_steps = len(train_loader)
print_every= len(train_loader) - 1
t1 = time.time()
Accuracies = []
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i+1) % print_every == 0:
            Accuracies.append(round((correct/total)*100, 2))
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Epoch [1/10], Step [140/141], Loss: 0.4076, Accuracy: 90.62%
Epoch [2/10], Step [140/141], Loss: 0.4666, Accuracy: 78.12%
Epoch [3/10], Step [140/141], Loss: 0.1024, Accuracy: 100.00%
Epoch [4/10], Step [140/141], Loss: 0.0659, Accuracy: 100.00%
Epoch [5/10], Step [140/141], Loss: 0.2355, Accuracy: 93.75%
Epoch [6/10], Step [140/141], Loss: 0.2282, Accuracy: 93.75%
Epoch [7/10], Step [140/141], Loss: 0.4024, Accuracy: 81.25%
Epoch [8/10], Step [140/141], Loss: 0.3880, Accuracy: 81.25%
Epoch [9/10], Step [140/141], Loss: 0.1283, Accuracy: 96.88%
Epoch [10/10], Step [140/141], Loss: 0.4201, Accuracy: 84.38%
######## Training Finished in 231.16344213485718 seconds ###########


In [9]:
# Calculating Test Accuracy
model.eval() 

with torch.no_grad(): 
    correct = 0
    total = 0
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))

Test Accuracy of the model on the 1127 test images: 72.67080745341616 %


In [18]:
# total_step = len(train_loader)
# import time
# print_every = len(train_loader) - 1
# loss_values = []
# start_time = time.time()
# total_step = len(train_loader)
# epoch_times = []

# # Training 
# for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(train_loader): 
#         epoch_start_time = time.time()
#         running_loss = 0.0
#         images = images.to(device)
#         labels = labels.to(device)
        
#         outputs = model(images)
#         loss = criterion(outputs, labels)
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item() 
#         if (i+1) % print_every == 0:
#             epoch_times.append(time.time() - epoch_start_time)
#             loss_values.append(running_loss / print_every)
#             print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}: Batch Loss : {}' 
#                    .format(epoch+1, num_epochs, i+1, total_step, loss.item(), running_loss / print_every))
#             running_loss = 0
# end_of_training = time.time() - start_time



# arr_pred = np.empty((0,len(test_dataset)), int)

# arr_label = np.empty((0,len(test_dataset)), int)

# # Prediction
# with torch.no_grad(): 
#     correct = 0
#     total = 0
#     for images, labels in test_loader: 
#         images = images.to(device)
#         labels = labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
        
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         pred = predicted.cpu().numpy()
#         lb = labels.cpu().numpy()
#         arr_pred = np.append(arr_pred, pred)
#         arr_label = np.append(arr_label, lb)
        
#     print('Test Accuracy of the model on the ' + str(len(test_dataset)) + ' test images: {} %'.format(100 * correct / total))